# Langchain

In [1]:
!pip install --upgrade --quiet langchain langchain_community langchain-anthropic langchain_experimental pypdf huggingface_hub sentence-transformers chromadb docarray

## Load data

In [4]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf"),
    PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture04.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

## Splitting

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

199

## VectorStore & Embedding

In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


# save to disk
db1 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")

# load from disk
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
print(docs[0].page_content)

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# DEFAULT_SENTENCE_EMBEDDING_MODEL = 'intfloat/multilingual-e5-base'
SENTENCE_EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

persist_directory = 'chromadb'

chromadb = Chroma.from_documents(
    docs,
    embedding = HuggingFaceEmbeddings(model_name=SENTENCE_EMBEDDING_MODEL),
    persist_directory=persist_directory
)

## Retrieval

### Similarity Search

In [66]:
question = "what did they say about matlab?"
docs_ss = chromadb.similarity_search(question,k=3)

In [67]:
print(f'1: {docs_ss[0].page_content[:100]}')
print(f'2: {docs_ss[1].page_content[:100]}')

1: those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 
2: those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 


### Max Marginal Relevance search (MMR)

The idea of MMR is that you may not always want to choose the most similar responses.
Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

In [68]:
docs_mmr = chromadb.max_marginal_relevance_search(question,k=3)

In [69]:
print(f'1: {docs_mmr[0].page_content[:100]}')
print(f'2: {docs_mmr[1].page_content[:100]}')

1: those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 
2: And as an aside, this algorithm I just showed you, it seems like it must be a pretty 
complicated al


In [70]:
question = "what did they say about regression in the third lecture?"
docs = chromadb.similarity_search(question, k=3)

for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300]+'\n')

0: MachineLearning-Lecture03  
Instructor (Andrew Ng) :Okay. Good morning and welcome b ack to the third lecture of 
this class. So here’s what I want to do t oday, and some of the topics I do today may seem 
a little bit like I’m jumping, sort  of, from topic to topic, but here’s, sort of, the outline

0: MachineLearning-Lecture03  
Instructor (Andrew Ng) :Okay. Good morning and welcome b ack to the third lecture of 
this class. So here’s what I want to do t oday, and some of the topics I do today may seem 
a little bit like I’m jumping, sort  of, from topic to topic, but here’s, sort of, the outline

2: Instructor (Andrew Ng) :All right, so who thought driving could be that dramatic, right? 
Switch back to the chalkboard, please. I s hould say, this work was done about 15 years 
ago and autonomous driving has come a long way. So many of you will have heard of the 
DARPA Grand Challenge, where one o



In [71]:
for doc in docs:
    print(doc.metadata)

{'page': 0, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf'}
{'page': 2, 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture02.pdf'}


### Other types of retrieval

In [ ]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load PDF
loader = PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [ ]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes")

In [ ]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

## Question-Answering

### Claude3

In [ ]:
import os
from google.colab import userdata

# set key to ENV
key = 'ANTHROPIC_API_KEY'
os.environ[key] = userdata.get(key)

In [59]:
from langchain_anthropic import ChatAnthropic

chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

### RetrievalQA chain

In [60]:
# RetrievalQA chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    chat,
    retriever=chromadb.as_retriever(),
)

In [61]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

"Yes, probability is an important prerequisite topic for this machine learning class. The instructor mentions that he assumes students have familiarity with basic probability and statistics, such as knowing what random variables, expectation, and variance are. \n\nHe says most undergraduate statistics classes, like Stat 116 at Stanford, will provide more than enough probability background for this course. He also mentions that for students who haven't seen probability material in a while, some of the discussion sections will review the probability prerequisites as a refresher."

### Prompt

In [62]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [63]:
# RetrievalQA chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    chat,
    retriever=chromadb.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [64]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a topic covered in this class. The instructor mentions that if students need a refresher on the foundations of probability, the discussion sections taught by the TAs will review probability. Thanks for asking!'